In [1]:
#!add-apt-repository -y ppa:jonathonf/ffmpeg-4 && apt update && apt install -y ffmpeg
#!pip install --quiet datasets git+https://github.com/huggingface/transformers evaluate huggingface_hub jiwer

In [2]:
#!git config --global credential.helper store

In [1]:
from huggingface_hub import login

login()

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "mozilla-foundation/common_voice_11_0", "el", revision="streaming", split="test", streaming=True, use_auth_token=True
)

In [3]:
from transformers import pipeline

whisper_asr = pipeline(
    "automatic-speech-recognition", model="farsipal/whisper-small-el", device=-1 # GPU seq ID
)

whisper_asr.model.config.forced_decoder_ids = whisper_asr.tokenizer.get_decoder_prompt_ids(language="el", task="transcribe")
#whisper_asr.model.config.forced_decoder_ids = whisper_asr.tokenizer.get_decoder_prompt_ids(language="el", task="translate")

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [4]:
whisper_norm = whisper_asr.tokenizer._normalize

def get_text(sample):
    if "text" in sample:
        return sample["text"]
    elif "sentence" in sample:
        return sample["sentence"]
    elif "normalized_text" in sample:
        return sample["normalized_text"]
    elif "transcript" in sample:
        return sample["transcript"]
    else:
        raise ValueError(f"Sample: {sample.keys()} has no transcript.")

def normalise(batch):
    batch["norm_text"] = whisper_norm(get_text(batch))
    return batch

In [5]:
from datasets import Audio

dataset = dataset.take(10)

# resample to 16kHz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# normalise references
dataset = dataset.map(normalise)

In [6]:
for data in dataset:
    print(whisper_asr(data["audio"]))

Reading metadata...: 1696it [00:00, 4757.90it/s]
/home/farsipal/projects/GITHUB/transformers/src/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Γιατί την αλυσίδα μου την είχε δώσει εμένα;', 'path': ['common_voice_el_27443549.mp3']}
{'text': ' Από το φεγγάρι πρέπει να μας έπεσε σε εσύ.', 'path': ['common_voice_el_24677613.mp3']}
{'text': ' μη θέλοντας να πιστέψουν ο γέρος εξακολύθησε.', 'path': ['common_voice_el_25653813.mp3']}
{'text': ' Ήπε βασιλιάς η εικόντας ακόμα πισιλότερο το βασιλικό του κεφάλι.', 'path': ['common_voice_el_21241420.mp3']}
{'text': ' και άλλη στρατιώτας που είναι.', 'path': ['common_voice_el_31386842.mp3']}
{'text': ' Ήπε ανήσυχα', 'path': ['common_voice_el_32697832.mp3']}
{'text': ' "Ελα εδώ και εσύ", είπε το δεύτερο υπασπιστή.', 'path': ['common_voice_el_29009633.mp3']}
{'text': ' Ποπωραντε εγώ', 'path': ['common_voice_el_20740959.mp3']}
{'text': ' και βρήκε το δεσμοφύλακα που κουβεδιαζε.', 'path': ['common_voice_el_27424634.mp3']}
{'text': ' Δεν κάνεις λοιπόν τότε εκείνο το οποίο θέλεις.', 'path': ['common_voice_el_23970440.mp3']}
